In [ ]:
import os
import json
import tensorflow as tf
import shap
from sklearn.metrics import mean_squared_error, make_scorer
from skopt import BayesSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from Tools.data_model import *
from Tools.data_process import (
    load_json_data,
    X_Y_split,
    scale_data,
    inverse_transform_2d,
    search_space,
)

""" Load configuration """
configs = json.load(open(os.path.join("configs", "model.json"), "r"))
data_info = configs["data"]
json_index = data_info["json_index"]

""" Load training and validation data """
file_path = os.path.join("data", data_info["filename"])
data_trains = load_json_data(file_path, "prediction_train", json_index)
data_validations = load_json_data(file_path, "prediction_validation", json_index)


""" Preprocess data """
scaler, train_scaled, validation_scaled = scale_data(data_trains, data_validations)

""" Set time steps """
n_past = configs["data"]["n_past"]
n_future = configs["data"]["n_future"]
train_X, train_Y = X_Y_split(train_scaled, n_past, n_future)
validation_X, validation_Y = X_Y_split(validation_scaled, n_past, n_future)

""" Model parameters and creation """
input_shape = train_X[0].shape
params = configs["training_params"]
model_type = configs["model"]["modeltype"]
create_model = eval(f"create_{model_type}_model")

""" Hyperparameter optimization """
model = KerasRegressor(build_fn=create_model, input_shape=input_shape, n_future=n_future, verbose=1)
def positive_mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)
mse_scorer = make_scorer(positive_mse, greater_is_better=False)

class FitnessCallback:
    def __init__(self):
        self.scores = []
    def __call__(self, res):
        self.scores.append(res.fun)
fitness_callback = FitnessCallback()

params_converted = search_space(params)
search = BayesSearchCV(estimator=model, search_spaces=params_converted, cv=2, n_iter=50, scoring=mse_scorer)
search.fit(train_X, train_Y, callback=fitness_callback)
best_model = search.best_estimator_.model
best_params = search.best_params_
fitness = fitness_callback.scores

""" Model prediction """
prediction = best_model.predict(validation_X)

""" Inverse scaling """
col = train_scaled.shape[1]
pred = inverse_transform_2d(scaler, prediction, col)
test = inverse_transform_2d(scaler, validation_Y, col)

""" Model interpretability """
shap.initjs()
shap_explainer = shap.DeepExplainer(best_model, train_X)
shap_values = shap_explainer.shap_values(validation_X)

""" Hybrid Model Prediction """
data_set = load_json_data(file_path, "data_set")
hybrid_model = HybridModel(best_model, data_set)